<a href="https://colab.research.google.com/github/Haniehnamavari/telegram_bot/blob/main/mnist_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt


(x_train, _), (x_test, _) = mnist.load_data()
x_data = np.concatenate((x_train, x_test), axis=0)
x_data = x_data.astype('float32') / 255.
x_data = np.reshape(x_data, (x_data.shape[0], 28, 28, 1))


noise_factor = 0.5
x_noisy = x_data + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_data.shape)
x_noisy = np.clip(x_noisy, 0., 1.)


split_index = int(0.8 * len(x_data))
x_train_noisy = x_noisy[:split_index]
x_train_clean = x_data[:split_index]
x_test_noisy = x_noisy[split_index:]
x_test_clean = x_data[split_index:]


input_img = Input(shape=(28, 28, 1))


x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)


x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()


autoencoder.fit(x_train_noisy, x_train_clean,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test_clean))


x_test_denoised = autoencoder.predict(x_test_noisy)


(_, y_train), (_, y_test) = mnist.load_data()
y_data = np.concatenate((y_train, y_test), axis=0)
y_train_cls = y_data[:split_index]
y_test_cls = y_data[split_index:]


classifier_input = Input(shape=(28, 28, 1))
x = Conv2D(32, (3, 3), activation='relu')(classifier_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

classifier = Model(classifier_input, output)
classifier.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
classifier.summary()


classifier.fit(x_train_clean, y_train_cls,
               epochs=5,
               batch_size=128,
               validation_data=(x_test_denoised, y_test_cls))


loss, acc = classifier.evaluate(x_test_denoised, y_test_cls)
print("Classifier Accuracy on Denoised Images:", acc)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 16)       │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 14, 14, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 14, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 1)      │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,193 (47.63 KB)

 Trainable params: 12,193 (47.63 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121,930 (476.29 KB)

 Trainable params: 121,930 (476.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 44s 97ms/step - accuracy: 0.8236 - loss: 0.5947 - val_accuracy: 0.9474 - val_loss: 0.1657
Epoch 2/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 79s 91ms/step - accuracy: 0.9783 - loss: 0.0712 - val_accuracy: 0.9559 - val_loss: 0.1455
Epoch 3/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 42s 95ms/step - accuracy: 0.9848 - loss: 0.0470 - val_accuracy: 0.9603 - val_loss: 0.1288
Epoch 4/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 80s 91ms/step - accuracy: 0.9888 - loss: 0.0359 - val_accuracy: 0.9605 - val_loss: 0.1299
Epoch 5/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 40s 90ms/step - accuracy: 0.9905 - loss: 0.0284 - val_accuracy: 0.9562 - val_loss: 0.1488
438/438 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9525 - loss: 0.1637
Classifier Accuracy on Denoised Images: 0.9562143087387085
